Load Trainingsfile:

In [21]:
import os
import torch
import numpy as np
import tqdm
from radioNN.data.loader import AntennaDataset

from radioNN.networks.antenna_resnet_network import AntennaNetworkResNet
from radioNN.networks.antenna_cnn_network import AntennaNetworkCNN
from radioNN.networks.antenna_fc_network import AntennaNetworkFC
from radioNN.networks.antenna_skipfc_network import AntennaNetworkSkipFC

from radioNN.process_network import NetworkProcess

import scipy.interpolate as intp
from radiotools.analyses import energy_fluence
import matplotlib.pyplot as plt

In [22]:

radio_data_path = "/Volumes/DenisDRIVE/BACHELORTHESIS/memmap_files/memmap"
memmap_mode = "r"
if not os.path.exists(radio_data_path):
    radio_data_path = ""
    memmap_mode = "r"
    
assert os.path.exists(radio_data_path)
input_data_file = os.path.join(radio_data_path, "input_data.npy")
input_meta_file = os.path.join(radio_data_path, "meta_data.npy")
antenna_pos_file = os.path.join(
    radio_data_path, "antenna_pos_data.npy"
)
output_meta_file = os.path.join(
    radio_data_path, "output_meta_data.npy"
)
output_file = os.path.join(radio_data_path, "output_gece_data.npy")
output_file = os.path.join(radio_data_path, "output_vBvvB_data.npy")

process = NetworkProcess(model_class=AntennaNetworkSkipFC,
                       # one_shower=one_shower, 
                        percentage=80,
                        batch_size=8, wb=False,
                         )

dataset = AntennaDataset(
    input_data_file,
    input_meta_file,
    antenna_pos_file,
    output_meta_file,
    output_file,
    mmap_mode=memmap_mode,
    one_shower=33,
    #percentage=0.01,
)


Using the data from /Volumes/DenisDRIVE/BACHELORTHESIS/memmap_files/memmap in cpu with memmap mode: r using 80% of data
Total showers after filter 21524


100%|██████████| 17219/17219 [00:15<00:00, 1089.02it/s]


2
Using <class 'radioNN.networks.antenna_skipfc_network.AntennaNetworkSkipFC'>


In [23]:
(inp_data_real, inp_meta_real, antenna_pos_real, outp_meta_real, outp_data_real) = dataset.data_of_single_shower(0)

In [26]:
event_data = torch.Tensor([inp_data_real])
meta_data_inp = torch.Tensor([inp_meta_real])
antenna_pos = torch.Tensor([antenna_pos_real])
output_meta = torch.Tensor([outp_meta_real])
output_real = torch.Tensor([outp_data_real])

#meta_data_inp.T[2] /= 700
##  3: density at xmax - Useless, barely varies in value
##  4: Height(xmax) - from 1% data, we see div is more sensible than log
#meta_data_inp.T[4] /= 7e5
##  5: Eem
#meta_data_inp.T[5] = torch.log(meta_data_inp.T[5] + 1e-14) / 20
## 10: PRMPAR
## No nothing
## TODO: (Think about what to do for the 5000 value outlier)
## 11: Primary Energy
#meta_data_inp.T[11] = torch.log(meta_data_inp.T[11] + 1e-14)
meta_data_inp = meta_data_inp.T
event_data_t,meta_data_inp_t,antenna_pos_t,output_meta_t,output_real_t = process.dataset.transform(event_data,meta_data_inp,antenna_pos,output_meta,output_real)
pred_output_meta, pred_output = process.model(event_data_t,meta_data_inp_t,antenna_pos_t)
pred_output = pred_output.detach().numpy()

IndexError: index 2 is out of bounds for dimension 0 with size 1

In [ ]:
process.model = torch.load("/Volumes/DenisDRIVE/BACHELORTHESIS/runs_with_all_data/2503Mar30Sun_213729/SavedModel", map_location=torch.device('cpu'))
process.model.eval()

print(inp_data_real)
print(inp_meta_real)
print(antenna_pos_real)

output = process.model(torch.tensor([inp_data_real]), torch.tensor([inp_meta_real]), torch.tensor([antenna_pos_real]))

print(output.shape)

[[0.         0.02310491 0.03662041 ... 0.23068905 0.23072194 0.2307548 ]
 [0.26585695 0.26995242 0.26916918 ... 0.6268167  0.62664634 0.62647635]
 [0.13964702 0.14504227 0.13331681 ... 0.52298737 0.5228203  0.5226812 ]
 ...
 [0.38752973 0.3872697  0.3870077  ... 0.36981222 0.36980855 0.36980483]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.21786498 0.21786875 0.21787249 ... 0.25388008 0.2538801  0.25388014]]
[ 2.0000000e+00 -8.6484647e-01  9.4647145e-01  2.0314297e+03
  9.3156241e-02  1.9407213e+00  9.3043649e-01  6.7809708e+01
  0.0000000e+00  4.9247536e-01  1.4000000e+01  1.8243923e+01
  1.5578609e+00]
[[ 3.92976999e-01  3.86270255e-01 -3.14564686e+01]
 [ 4.08343673e-01 -8.66299748e-01  2.12779274e+01]
 [ 4.07079250e-01 -6.21281207e-01  1.68625164e+01]
 [ 4.01718676e-01 -3.74181211e-01 -1.70519626e+00]
 [ 3.95319402e-01 -1.23250306e-01 -2.35484982e+01]
 [ 3.91662985e-01  1.31101519e-01 -3.58718796e+01]
 [ 3.98455173e-01  6.39091194e-01 -1.28887548e+01]

RuntimeError: Tensors must have same number of dimensions: got 2 and 3